In [1]:
!pip install fasta_reader

In [1]:
from fasta_reader import read_fasta
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
from torch.utils.data import random_split
from torch.utils.data import DataLoader



In [11]:
#Functions


def count_cg(dna_sequence: str) -> int:

    """ This function iterates through a DNA sequence string to count occurrences of the "CG" dinucleotide.
    It's a straightforward approach that does not require any external libraries."""

    cg_count = 0

    # Iterate over the sequence to find 'CG'
    for i in range(len(dna_sequence) - 1):  # Subtract 1 to avoid index out of range
        if dna_sequence[i:i+2] == "CG":  # Check if the current and next character form a 'CG' pair
            cg_count += 1

    return cg_count



def sequence_into_chunks_loop(sequence: str, chunk_size=800, sequence_size=128)-> list:
    """This function divides a long DNA sequence into smaller chunks of a specified size
    (default is 128). It's useful for processing large sequences in manageable pieces."""

    chunks = []
    for i in range(0,chunk_size*sequence_size, sequence_size):
        chunks.append(sequence[i:i + sequence_size])
    return chunks


def create_dataset_with_cg_counts(sequence: str, chunk_size=800, sequence_size=128) -> list:

    """Utilizes sequence_into_chunks_loop to first slice the sequence into chunks and then applies count_cg to each chunk to
  create a dataset where each item is a tuple containing a chunk and its corresponding "CG" count."""
    # Use the provided function to slice the sequence into chunks
    chunks = sequence_into_chunks_loop(sequence, chunk_size,sequence_size)

    # Create a new dataset by counting 'CG' in each chunk
    # Each element in the new dataset is a tuple (chunk, cg_count)
    dataset = [(chunk, count_cg(chunk)) for chunk in chunks]

    return dataset

vocab = ['N', 'A', 'C', 'G', 'T']  # Include 'N' for unknown nucleotides
vocab_size = len(vocab)
char_to_index = {char: idx for idx, char in enumerate(vocab)}


def sequence_to_indices(sequence:str, char_to_index):

    """Converts a DNA sequence into a list of numerical indices based on a predefined mapping (char_to_index).
    This step is crucial for transforming categorical nucleotide data into a numerical format that machine learning models
    can process."""

    return [char_to_index[char] for char in sequence]


def sequence_to_indices_chunk(sequence:str, chunk_size:int,sequence_size:int,char_to_index)->list :

    """ convert sequence to mapped indices"""
    chunks = sequence_into_chunks_loop(sequence, chunk_size,sequence_size)

    chunk_indices=[]
    for i in range(len(chunks)):
      row=sequence_to_indices(chunks[i],char_to_index)
      chunk_indices.append(list(row))
    return chunk_indices

def sequence_to_indeces_dataset(sequence: str, chunk_size=800, sequence_size=128)-> list:
    """convert a sequence into a dataset that has label appended in the end """
    chunks = sequence_into_chunks_loop(sequence, chunk_size,sequence_size)
    chunks_ind= sequence_to_indices_chunk (cleaned_sequence,chunk_size,sequence_size,char_to_index)
    dataset=[]
    for i in range(chunk_size):
      dataset+=[(chunks_ind[i],count_cg(chunks[i]))]
    return dataset


#pad the sequence and convert to tensor

def pad_and_retrieve_counts(dataset):
    """
    Pad the sequences represented as lists of indices to uniform length and retrieve CG counts.

    Args:
    - dataset (list of tuples): Each tuple contains a sequence as a list of indices and its "CG" count.

    Returns:
    - Tensor: Padded sequences.
    - Tensor: Corresponding "CG" counts.
    """
    # Unpack sequences and counts from the dataset
    sequence_tensors = [torch.tensor(seq) for seq, _ in dataset]
    cg_counts = torch.tensor([count for _, count in dataset])

    # Pad the sequences
    padded_sequences = pad_sequence(sequence_tensors, batch_first=True)

    return padded_sequences, cg_counts




In [12]:
FILE ="/content/drive/MyDrive/sequence.fasta"
for item in read_fasta(FILE):
  continue
sequence=item.sequence

In [13]:
cleaned_sequence=''.join(filter(lambda char: char in ('A', 'C', 'G', 'T'), sequence))
dataset_i=sequence_to_indeces_dataset(cleaned_sequence)
padded_sequences,cg_counts=pad_and_retrieve_counts(dataset_i)

Preprocess the Data for Neural Networks


In [14]:
#TRAINING SET , VALIDATION SET , TEST SET
total_samples = 800 # Should be 500 based on your description
train_size = 400
val_size = 200
test_size = 200  # Total = 500

dataset = TensorDataset(padded_sequences, cg_counts.float())

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

batch_size_train = 1  # Example batch size for training; adjust as needed
batch_size_val_test = 1  # Batch size for validation and testing

train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size_val_test, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size_val_test, shuffle=False)


Model Prep

In [15]:
class DNALSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim=1):
        super(DNALSTM, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(in_features=hidden_dim, out_features=output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        # You might want to take the output from the last timestep or aggregate outputs
        final_feature_map = lstm_out[:, -1, :]
        output = self.fc(final_feature_map)
        return output


In [16]:
def train_model__(model, data_loader, loss_function, optimizer, device, num_epochs=100):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        total_loss = 0.0
        num_batches = 0

        for sequences, labels in data_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(sequences).squeeze()  # Adjust based on your model's output
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()  # Accumulate the loss
            num_batches += 1

        # Calculate the average loss over all batches
        avg_loss = total_loss / num_batches
        print(f'Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.4f}')


In [17]:
def evaluate_model(model, data_loader, device):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    with torch.no_grad():  # No need to track gradients during evaluation
        for sequences, labels in data_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            predictions = model(sequences)
            loss = loss_function(predictions, labels.float())  # Assuming regression; use .float() for labels if needed
            total_loss += loss.item()

    avg_loss = total_loss / len(data_loader)  # Calculate average loss
    print(f"Average Loss: {avg_loss:.4f}")

In [18]:
#Hyperparameters
vocab_size = len(vocab)
embedding_dim = 248
hidden_dim = 128
output_dim = 1  # Assuming a regression task; adjust as needed for classification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DNALSTM(vocab_size, embedding_dim, hidden_dim, output_dim)
model = model.to(device)
loss_function = nn.MSELoss()  # For regression
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)


In [19]:
#Train
train_model__(model, train_loader, loss_function, optimizer,"cuda")


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/100, Average Loss: 6.0266
Epoch 2/100, Average Loss: 5.3361
Epoch 3/100, Average Loss: 4.8609
Epoch 4/100, Average Loss: 4.3229
Epoch 5/100, Average Loss: 3.9718
Epoch 6/100, Average Loss: 3.7731
Epoch 7/100, Average Loss: 3.7577
Epoch 8/100, Average Loss: 3.6823
Epoch 9/100, Average Loss: 3.7130
Epoch 10/100, Average Loss: 3.6933
Epoch 11/100, Average Loss: 2.9640
Epoch 12/100, Average Loss: 2.4298
Epoch 13/100, Average Loss: 2.2809
Epoch 14/100, Average Loss: 2.0299
Epoch 15/100, Average Loss: 1.9565
Epoch 16/100, Average Loss: 1.5955
Epoch 17/100, Average Loss: 1.4113
Epoch 18/100, Average Loss: 1.4297
Epoch 19/100, Average Loss: 1.6051
Epoch 20/100, Average Loss: 1.8878
Epoch 21/100, Average Loss: 1.5462
Epoch 22/100, Average Loss: 1.4591
Epoch 23/100, Average Loss: 1.3893
Epoch 24/100, Average Loss: 1.1796
Epoch 25/100, Average Loss: 1.1997
Epoch 26/100, Average Loss: 0.9650
Epoch 27/100, Average Loss: 1.1245
Epoch 28/100, Average Loss: 0.7869
Epoch 29/100, Average Loss: 0

In [22]:
#Save
torch.save(model.state_dict(), "/content/drive/MyDrive/cpg_detector_lstm.pth")


Evaluate model


In [20]:
evaluate_model(model, test_loader, device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Average Loss: 1.1885


In [21]:
evaluate_model(model, val_loader, device)

Average Loss: 0.6296


LOAD MODEL HERE

In [25]:
model_new = DNALSTM(vocab_size, embedding_dim, hidden_dim, output_dim)
model_new.load_state_dict(torch.load("/content/drive/MyDrive/cpg_detector_lstm.pth"))
model_new.to(device)


DNALSTM(
  (embedding): Embedding(5, 248)
  (lstm): LSTM(248, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)

In [26]:
evaluate_model(model_new, test_loader, device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Average Loss: 1.1885
